In [2]:
import torch
import torch.nn as nn

In [26]:
weight = 0.6
bias = 0.2

start = 0
end = 2
step = 0.03
X = torch.arange(start,end,step).unsqueeze(dim=1)
y = weight*X + bias

X[0:10]

tensor([[0.0000],
        [0.0300],
        [0.0600],
        [0.0900],
        [0.1200],
        [0.1500],
        [0.1800],
        [0.2100],
        [0.2400],
        [0.2700]])

In [27]:
train_split = int(len(X)*0.8)
X_train,y_train = X[:train_split],y[:train_split]
X_test,y_test = X[train_split:],y[train_split:]

len(X_train),len(y_train),len(X_test),len(y_test)

(53, 53, 14, 14)

In [28]:
class LinearRegression(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(1,
                                                requires_grad=True,
                                                dtype=torch.float32))
        self.bias = nn.Parameter(torch.randn(1,
                                                requires_grad=True,
                                                dtype=torch.float32))
    
    def forward(self,x:torch.Tensor):
        return self.weights * x + self.bias
    
    
    

In [29]:
torch.manual_seed(42)

model = LinearRegression()

list(model.parameters()),model.state_dict()

([Parameter containing:
  tensor([0.3367], requires_grad=True),
  Parameter containing:
  tensor([0.1288], requires_grad=True)],
 OrderedDict([('weights', tensor([0.3367])), ('bias', tensor([0.1288]))]))

In [30]:
with torch.inference_mode():
    y_preds = model(X_test)
y_preds

tensor([[0.6641],
        [0.6742],
        [0.6843],
        [0.6944],
        [0.7045],
        [0.7147],
        [0.7248],
        [0.7349],
        [0.7450],
        [0.7551],
        [0.7652],
        [0.7753],
        [0.7854],
        [0.7955]])

In [31]:
loss_fn = nn.L1Loss()
optimizer = torch.optim.SGD(params=model.parameters(),
                            lr = 0.01)


In [32]:
epochs = 10

for epoch in range(epochs):
    model.train()
    
    #forward pass
    y_pred = model(X_train)
    
    #loss fn
    loss = loss_fn(y_pred,y_train)
    
    #optimizer zero grad
    optimizer.zero_grad()
    
    #backpropagation
    loss.backward()
    
    
    optimizer.step()
    
    model.eval()
    
    print(model.state_dict())

OrderedDict([('weights', tensor([0.3445])), ('bias', tensor([0.1388]))])
OrderedDict([('weights', tensor([0.3523])), ('bias', tensor([0.1488]))])
OrderedDict([('weights', tensor([0.3601])), ('bias', tensor([0.1588]))])
OrderedDict([('weights', tensor([0.3679])), ('bias', tensor([0.1688]))])
OrderedDict([('weights', tensor([0.3757])), ('bias', tensor([0.1788]))])
OrderedDict([('weights', tensor([0.3835])), ('bias', tensor([0.1888]))])
OrderedDict([('weights', tensor([0.3913])), ('bias', tensor([0.1988]))])
OrderedDict([('weights', tensor([0.3991])), ('bias', tensor([0.2088]))])
OrderedDict([('weights', tensor([0.4069])), ('bias', tensor([0.2181]))])
OrderedDict([('weights', tensor([0.4146])), ('bias', tensor([0.2265]))])


Epoch: 0 | MAE Train Loss: 0.12096777558326721 | MAE Test Loss: 0.28253963589668274 
Epoch: 10 | MAE Train Loss: 0.0480034314095974 | MAE Test Loss: 0.12227609008550644 
Epoch: 20 | MAE Train Loss: 0.033110689371824265 | MAE Test Loss: 0.06758377701044083 
Epoch: 30 | MAE Train Loss: 0.02398909255862236 | MAE Test Loss: 0.04733004793524742 
Epoch: 40 | MAE Train Loss: 0.01490764506161213 | MAE Test Loss: 0.029083916917443275 
Epoch: 50 | MAE Train Loss: 0.005827839020639658 | MAE Test Loss: 0.009833999909460545 
Epoch: 60 | MAE Train Loss: 0.0065588983707129955 | MAE Test Loss: 0.015066231600940228 
Epoch: 70 | MAE Train Loss: 0.0065588983707129955 | MAE Test Loss: 0.015066231600940228 
Epoch: 80 | MAE Train Loss: 0.0065588983707129955 | MAE Test Loss: 0.015066231600940228 
Epoch: 90 | MAE Train Loss: 0.0065588983707129955 | MAE Test Loss: 0.015066231600940228 
Epoch: 100 | MAE Train Loss: 0.0065588983707129955 | MAE Test Loss: 0.015066231600940228 
Epoch: 110 | MAE Train Loss: 0.00655

In [10]:
import torch
import torch.nn as nn
import numpy as np

class LinearRegression:
    def __init__(self):
        self.model = None
    
    def fit(self,X,y,learning_rate = 0.001,epochs=1000):
        X = torch.tensor(X,dtype=torch.float32)
        y = torch.tensor(y,dtype=torch.float32).view(-1,1)
        
        self.model = nn.Linear(X.shape[1],1)
        
        loss_fn = nn.MSELoss()
        optimizer = torch.optim.SGD(self.model.parameters(),lr = learning_rate)
        
        for epoch in range(epochs):
            y_pred = self.model(X)
            loss = loss_fn(y_pred,y)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            if (epoch + 1) % 100 == 0:
                print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')
        
        
    def predict(self,X):
        X = torch.tensor(X,dtype=torch.float32)
        
        with torch.no_grad():
            y_pred = self.model(X)
            
        return y_pred.numpy()
    
if __name__ == "__main__":
    X = np.array([[1], [2], [3], [4]], dtype=np.float32)  
    y = np.array([1,2,3,4], dtype=np.float32)  
    # np.random.seed(0)
    # X = np.random.rand(100, 1) * 10  
    # y = 3.5 * X.squeeze() + np.random.randn(100) * 0.5 

    regressor = LinearRegression()
    regressor.fit(X, y, epochs=1000, learning_rate=0.01)

    # Make predictions
    X_test = np.array([[5], [6]], dtype=np.float32)
    predictions = regressor.predict(X_test)
    print("Predictions:", predictions)

Epoch [100/1000], Loss: 0.0194
Epoch [200/1000], Loss: 0.0107
Epoch [300/1000], Loss: 0.0058
Epoch [400/1000], Loss: 0.0032
Epoch [500/1000], Loss: 0.0018
Epoch [600/1000], Loss: 0.0010
Epoch [700/1000], Loss: 0.0005
Epoch [800/1000], Loss: 0.0003
Epoch [900/1000], Loss: 0.0002
Epoch [1000/1000], Loss: 0.0001
Predictions: [[5.016022]
 [6.023801]]
